In [1]:
%load_ext autoreload

In [2]:
%autoreload 2
import librosa
import os, json, glob
import matplotlib.pyplot as plt
import librosa, mir_eval
import numpy as np
from f0_analysis_utils import *

## Prepare data for training PT classifier
According to the annotations, split the mono audio segments into note events. 

Without candidate selection, 

directly classify the note events into:
* normal
* bend/release/prebend... (this will be further analyzed using signal processing)
* vibrato

directly classify the transitions (100 ms around the offset of note events) into:
* normal
* hammer/pull... (this will be further analyzed using signal processing)
* slide

Two classifiers, each distinguish between three classes.

### The code for generating the data matrix

In [ ]:
def extract_features(note_audio):
    return np.random.random((10,))
def extract_tran_features(tran_audio):
    return np.random.random((10,))

In [24]:
notes_data = [] # huge matrix with features and labels for note events (normal, bend/release, vibrato)
trans_data = [] # huge matrix with features and labels for note events (normal, hm/po, slide)

file = "Guns N' Roses - Welcome To The Jungle (ver 3)_Solo Guitar_0_0s.wav"
y, sr = librosa.load(os.path.join(FILTERED_AUDIO_DIR, file), sr=None)

anno_file = find_anno(file)
with open(anno_file) as anno:
    annotation = json.load(anno)
for i in range(len(annotation)):
    note_event = annotation[i]
    onset = note_event["time"]["start"]
    offset = onset + note_event["time"]["dur"]
    onset_sp = librosa.time_to_samples(onset, sr=sr)
    offset_sp = librosa.time_to_samples(offset, sr=sr)
    note_audio = y[onset_sp : offset_sp]
    feature = extract_features(note_audio)
    if note_event["effects"]["bend"]:
        label = 1
    elif note_event["effects"]["vibrato"]:
        label = 2
    else:
        label = 0
    feature_w_label = np.append(feature, label)
    notes_data.append(feature_w_label)

    # if the note event is the last one in the file, ignore its transitions 
    # because the transition won't be captured in the audio
    if i == len(annotation) - 1:
        break
    tran_onset_sp = offset_sp - librosa.time_to_samples(0.05, sr=sr)
    tran_offset_sp = offset_sp + librosa.time_to_samples(0.05, sr=sr)
    tran_audio = y[tran_onset_sp : tran_offset_sp]
    tran_feature = extract_tran_features(tran_audio)
    if note_event["effects"]["hammer"]:
        tran_label = 1
    elif note_event["effects"]["slide"]:
        tran_label = 2
    else:
        tran_label = 0
    tran_feature_w_label = np.append(tran_feature, tran_label)
    trans_data.append(tran_feature_w_label)

notes_data = np.array(notes_data)
trans_data = np.array(trans_data)

### Computing audio features
Frame-level features aggregated for the duration of the note event. 

Statistics: mean, std, max, min, skewness, kurtosis

All stats of spectral centroid, brightness, spread, skewness, kurtosis, flux, roll-off entropy, irregularity, roughness, inharmonicity, zero-crossing, low-energy ratio and their 1st order difference = 156

All stats of pitch and 1st order difference = 12

Mean and std of 40-MFCC and 1st order difference = 160

Run this part on colab

In [ ]:
def extract_features(note_audio):
    return np.random.random((10,))
def extract_tran_features(tran_audio):
    return np.random.random((10,))